In [1]:
!pip install m2cgen

     |████████████████████████████████| 73 kB 983 kB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import cv2
import os
from tqdm import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import random
import urllib
import requests
import webbrowser
import shutil
import collections
import re
import time
import json
from glob import glob
from PIL import Image
import pickle

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
import tensorflow.keras.layers as layers
from tensorflow.keras.initializers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import *

from sklearn.utils import *
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.metrics import *

import keras.backend as K
from scipy.stats import *
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb
import xgboost as xgb
import m2cgen as m2c

print('Using Tensorflow version:', tf.__version__)

Using Tensorflow version: 2.3.1


In [3]:
imgSize = 28
BS = 8
EPOCHS = 10
INIT_LR = 3e-4
opt = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [4]:
trainAug = ImageDataGenerator(rotation_range=5, rescale=1./255, validation_split=0.2)
validAug = ImageDataGenerator(rescale=1./255, validation_split=0.3)

In [5]:
train_generator = trainAug.flow_from_directory('../input/mnistasjpg/trainingSet/trainingSet/',
                                               target_size=(imgSize, imgSize),
                                               color_mode='grayscale',
                                               class_mode='categorical',
                                               batch_size=BS, 
                                               shuffle=False,
                                               subset='training')
valid_generator = validAug.flow_from_directory('../input/mnistasjpg/trainingSet/trainingSet/',
                                               target_size=(imgSize, imgSize),
                                               color_mode='grayscale',
                                               class_mode='categorical',
                                               batch_size=BS, 
                                               shuffle=False,
                                               subset='validation')

Found 33604 images belonging to 10 classes.
Found 12596 images belonging to 10 classes.


In [6]:
model = Sequential()
model.add(Flatten())

In [7]:
X_train = model.predict(train_generator, verbose = 1)
Y_train = train_generator.classes
X_valid = model.predict(valid_generator, verbose = 1)
Y_valid = valid_generator.classes

1575/1575 [==============================] - 29s 19ms/step


In [8]:
params = {
    'verbose': 0,
    'num_class':len(train_generator.class_indices.keys()),
    'boosting_type': 'gbdt',
    'min_data_in_bin':1,
    'max_bin':100,
    'num_leaves':5,
    'learning_rate':0.1,
    "n_estimators": 10
}
model = lgb.LGBMClassifier(**params)

In [9]:
model.fit(X_train, Y_train,
                eval_set=[(X_valid, Y_valid)],
                early_stopping_rounds=3)

[1]	valid_0's multi_logloss: 2.10718
Training until validation scores don't improve for 3 rounds
[2]	valid_0's multi_logloss: 1.96302
[3]	valid_0's multi_logloss: 1.84622
[4]	valid_0's multi_logloss: 1.7327
[5]	valid_0's multi_logloss: 1.63601
[6]	valid_0's multi_logloss: 1.55595
[7]	valid_0's multi_logloss: 1.47853
[8]	valid_0's multi_logloss: 1.40612
[9]	valid_0's multi_logloss: 1.34238
[10]	valid_0's multi_logloss: 1.28498
Did not meet early stopping. Best iteration is:
[10]	valid_0's multi_logloss: 1.28498


LGBMClassifier(max_bin=100, min_data_in_bin=1, n_estimators=10, num_class=10,
               num_leaves=5, verbose=0)

In [10]:
np.mean(model.predict(X_valid)==Y_valid)

0.7912829469672912

In [11]:
%%time
code = m2c.export_to_java(model)
len(code)

CPU times: user 168 ms, sys: 0 ns, total: 168 ms
Wall time: 168 ms


67346

In [12]:
with open('model.java', 'w') as f:
    f.write(code)